In [2]:
import torch
from transformers import RobertaModel, RobertaTokenizer

In [3]:
# Defining the model architecture
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 5)  # Assuming 5 classes

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [4]:
# Instantiating the model
model = RobertaClass()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Loading the model
model.load_state_dict(torch.load(r'C:\Users\Jash\OneDrive\Desktop\finalmodel2\pytorch_roberta_sentiment.bin', map_location=torch.device('cpu')))

<All keys matched successfully>

In [6]:
# Define your inference function to accept multiple inputs
def predict(texts):
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    inputs = tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,
        max_length=256,
        pad_to_max_length=True,
        return_token_type_ids=True,
        return_tensors='pt',
        truncation=True
    )
    with torch.no_grad():
        outputs = model(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'])
    _, predicted = torch.max(outputs, 1)
    return predicted.tolist()

In [7]:
# Test your model with multiple inputs'
texts_to_predict = ["check us out","hurry up, 50% off on every item", "10 people have already bought this, buy now","you have won 1000$ sign up now"] # giving manual inputs to test the trained model
predicted_labels = predict(texts_to_predict)
print("Predicted Labels:", predicted_labels)

# 0 are not a dark pattern
# 1 are dark pattern

c:\Users\raju_\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Predicted Labels: [0, 1, 1, 1]
